In [2]:
import torch
import time
import json
import pathlib
import argparse

from transformers import (
    # pipeline,
    AutoConfig,
    AutoModelForCausalLM,
    # AutoModel,
    LlamaForCausalLM,
    T5ForConditionalGeneration,
    AutoTokenizer,
    LlamaTokenizer,
)

/home/taosy/bin/miniconda3/envs/pt-39-6/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_id="meta-llama/Llama-2-7b-chat-hf"
config = AutoConfig.from_pretrained(model_id)
model=AutoModelForCausalLM.from_pretrained(model_id)

Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.82s/it]


In [4]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNo

In [ ]:
from torchinfo import summary
summary(model,depth=8,row_settings=["var_names","depth"])

In [ ]:
print(model)

In [ ]:
a=model.model.layers[0].self_attn.rotary_emb
print(a)
type(a)